<a href="https://colab.research.google.com/github/PrasunaA/LangChain1/blob/main/LangChain_HF_Motivator_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌟 LangChain Agentic AI Examples with Hugging Face FLAN-T5 (No API Key)
This notebook demonstrates two simple agentic AI use cases using LangChain and a local Hugging Face model (FLAN-T5):
- ✅ Generate motivational quotes for different professions
- ✅ Summarize short news updates into one sentence

In [1]:
# ✅ Install required libraries
!pip install transformers langchain huggingface_hub

In [2]:
# ✅ Imports
from transformers import pipeline
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import Generation, LLMResult
from langchain_core.pydantic_v1 import Field
from typing import Optional, List, Any
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# ✅ Load the Hugging Face FLAN-T5 model (no API key required)
generator = pipeline("text2text-generation", model="google/flan-t5-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
# ✅ Define LangChain-compatible wrapper
class HuggingFacePipelineLLM(LLM):
    pipeline: Any = Field(...)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        result = self.pipeline(prompt, max_length=256, do_sample=True)
        return result[0]["generated_text"]

    def _generate(self, prompt: str, stop: Optional[List[str]] = None) -> LLMResult:
        output_text = self._call(prompt, stop)
        return LLMResult(generations=[[Generation(text=output_text)]])

    @property
    def _llm_type(self) -> str:
        return "custom-hf-pipeline"

In [5]:
# ✅ Wrap the model
wrapped_llm = HuggingFacePipelineLLM(pipeline=generator)

## ✨ Example 1: Generate a Motivational Quote for a Specific Profession

In [6]:
motivation_prompt = PromptTemplate.from_template("Give me a motivational quote for {profession}.")
motivation_chain = LLMChain(llm=wrapped_llm, prompt=motivation_prompt)

response = motivation_chain.invoke({"profession": "software developers"})
print("Motivational Quote:\n", response["text"])

/tmp/ipython-input-6-4079450785.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  motivation_chain = LLMChain(llm=wrapped_llm, prompt=motivation_prompt)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Motivational Quote:
 if you can t code , you 'll never code .


## 📰 Example 2: Summarize a Short News Headline

In [7]:
summary_prompt = PromptTemplate.from_template("Summarize this in one sentence: {news}")
summary_chain = LLMChain(llm=wrapped_llm, prompt=summary_prompt)

news_text = "NASA's Mars rover has discovered new evidence of ancient riverbeds on the red planet, suggesting it once had a flowing water system."
response = summary_chain.invoke({"news": news_text})
print("Summary:\n", response["text"])

Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary:
 Scientists have discovered evidence of ancient riverbeds on Mars, suggesting that the red planet once had a flowing water system.
